In [ ]:
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification

# Load dataset
df = pd.read_excel('data/claims.xlsx')
print(df.head())

# Preprocessing
tokenizer = BertTokenizer.from_pretrained('model/bert-tensorflow2-answer-equivalence-bem-v1')
encodings = tokenizer(df['claim'].tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Labels (example: Fraud / Genuine claim)
labels = df['label'].values

# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((dict(encodings), labels)).batch(16)

# Load model
model = TFBertForSequenceClassification.from_pretrained(
    'model/bert-tensorflow2-answer-equivalence-bem-v1',
    num_labels=2
)

# Compile & Train
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(dataset, epochs=2)

# Save model
model.save_pretrained('saved_model')
tokenizer.save_pretrained('saved_model')
print('Model training complete and saved.')